In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy

In [2]:
class Node:
    def __init__(self, value):
        self.value = value
        self.neighbors = set()
        self.parent = None
        self.level = None
        self.visited = False
    
    #Retorna o índice do vértice quando for chamado
    def __repr__(self): 
        return str(self.value)
    
    #Adiciona um vértice adjacente
    def add_neighbor(self, w):
        self.neighbors.add(w)
    
    #Retorna o índice do vértice
    def get_value(self):
        return self.value
    
    #Retorna os vértices adjacentes
    def get_neighbors(self):
        return sorted(self.neighbors, key=lambda x: x.value)

In [3]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.matrix = np.zeros((n, n), dtype="object")
        self.nodes = np.empty(n, dtype="object")
        
        pbar = tqdm(range(n), desc="Adding edges...")
        
        for index in tqdm(range(n)):
            self.add_node(index + 1)
       

    def add_node(self, value):
        index = value - 1
        self.nodes[index] = Node(value)
    
    def add_edge(self, v_value, w_value):
        v_index = v_value - 1
        w_index = w_value - 1
        
        if v_value == w_value:
            self.matrix[v_index, v_index] = 2
            self.nodes[v_index].add_neighbor(self.nodes[v_index])
            
        else:
            self.matrix[v_index, w_index] = 1
            self.matrix[w_index, v_index] = 1
            self.nodes[v_index].add_neighbor(self.nodes[w_index])
            self.nodes[w_index].add_neighbor(self.nodes[v_index])
    
    def get_node(self, value):
        index = value - 1
        return self.nodes[index]
    
    def get_nodes(self):
        return self.nodes
    
    def get_node_edges(self):
        return {node.value:node.get_neighbors() for node in self.nodes}
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
    
    def get_lists(self):
        return [node.get_neighbors() for node in self.nodes]

In [4]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [18]:
folder = "inputs"
filename = "grafo_1.txt"

path = os.path.join(folder, filename)

In [19]:
graph = open_graph_txt(path)

Adding edges...:   0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [20]:
graph.get_node_edges()

{1: [1, 3, 16, 22, 31, 45, 46, 52, 82, 83, 84, 93],
 2: [2, 5, 11, 14, 19, 33, 37, 38, 49, 99],
 3: [1, 14, 36, 39, 52, 76, 80, 83, 85, 94, 95],
 4: [7, 15, 24, 26, 39, 71, 76, 84, 88, 92, 95, 98, 99],
 5: [2, 7, 14, 19, 23, 55, 59, 62, 65, 91, 94, 95, 98],
 6: [15, 22, 48, 50, 59, 62, 71, 79, 83, 91, 100],
 7: [4, 5, 13, 27, 33, 36, 43, 52, 56, 74, 84, 87, 90],
 8: [34, 40, 49, 50, 57, 58, 60, 65, 89],
 9: [10, 11, 15, 16, 21, 24, 46, 60, 65, 75, 89, 90, 99],
 10: [9, 20, 53, 72, 73, 82, 83, 90, 97],
 11: [2, 9, 14, 22, 23, 25, 46, 57, 80, 97],
 12: [13, 16, 45, 51, 55, 61, 70, 73, 76, 80, 85],
 13: [7, 12, 40, 48, 49, 58, 59, 81, 99],
 14: [2, 3, 5, 11, 16, 32, 45, 55, 64, 83],
 15: [4, 6, 9, 26, 34, 56, 65, 93, 94, 98, 100],
 16: [1, 9, 12, 14, 19, 25, 51, 60, 62, 89, 93, 94],
 17: [23, 27, 30, 34, 42, 61, 64, 72, 81, 82, 83, 89, 93, 94],
 18: [19, 28, 38, 44, 67, 69, 71, 72, 78, 86, 99],
 19: [2, 5, 16, 18, 27, 34, 35, 53, 69, 70, 86, 97, 100],
 20: [10, 22, 28, 31, 32, 42, 55, 74,

In [23]:
graph.get_lists()

[[1, 3, 16, 22, 31, 45, 46, 52, 82, 83, 84, 93],
 [2, 5, 11, 14, 19, 33, 37, 38, 49, 99],
 [1, 14, 36, 39, 52, 76, 80, 83, 85, 94, 95],
 [7, 15, 24, 26, 39, 71, 76, 84, 88, 92, 95, 98, 99],
 [2, 7, 14, 19, 23, 55, 59, 62, 65, 91, 94, 95, 98],
 [15, 22, 48, 50, 59, 62, 71, 79, 83, 91, 100],
 [4, 5, 13, 27, 33, 36, 43, 52, 56, 74, 84, 87, 90],
 [34, 40, 49, 50, 57, 58, 60, 65, 89],
 [10, 11, 15, 16, 21, 24, 46, 60, 65, 75, 89, 90, 99],
 [9, 20, 53, 72, 73, 82, 83, 90, 97],
 [2, 9, 14, 22, 23, 25, 46, 57, 80, 97],
 [13, 16, 45, 51, 55, 61, 70, 73, 76, 80, 85],
 [7, 12, 40, 48, 49, 58, 59, 81, 99],
 [2, 3, 5, 11, 16, 32, 45, 55, 64, 83],
 [4, 6, 9, 26, 34, 56, 65, 93, 94, 98, 100],
 [1, 9, 12, 14, 19, 25, 51, 60, 62, 89, 93, 94],
 [23, 27, 30, 34, 42, 61, 64, 72, 81, 82, 83, 89, 93, 94],
 [19, 28, 38, 44, 67, 69, 71, 72, 78, 86, 99],
 [2, 5, 16, 18, 27, 34, 35, 53, 69, 70, 86, 97, 100],
 [10, 22, 28, 31, 32, 42, 55, 74, 92, 96],
 [9, 29, 44, 50, 52, 58, 72, 82, 84, 94, 100],
 [1, 6, 11, 20

In [24]:
graph.get_matrix()

array([[2, 0, 1, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=object)

In [25]:
graph.get_matrix_beautiful()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,2,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,1,0
5,0,1,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
97,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
98,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    print("Número de arestas:", graph.get_matrix().sum()/2)
    print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
    print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
    print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
    print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))

In [27]:
graph_statistics(graph)

Número de vértices: 100
Número de arestas: 569.0
Grau mínimo: 5
Grau máximo: 16
Grau médio: 11.38
Mediana do Grau: 11.0


In [28]:
class Stack:
    
    def __init__(self, size):
        self.size = size
        self.top = -1
        
        self.values = np.empty(self.size, dtype="object")
    
    def is_full(self):
        return (self.top == self.size - 1)
    
    def is_empty(self):
        return (self.top == -1)
    
    def stack_up(self, node):
        if self.is_full():
            print("Stack is full")
        else:
            self.top += 1
            self.values[self.top] = node
    
    def unstack(self):
        if self.is_empty():
            print("Stack is empty")
        else:
            temp = self.values[self.top]
            self.top -= 1
            return temp
    
    def get_top(self):
        if self.top != -1:
            return self.values[self.top]
        else:
            return -1
        

In [29]:
class DFS:
    def __init__(self, graph, root):
        self.graph = copy.deepcopy(graph)
        self.root = self.graph.nodes[root-1]
        self.root.level = 0
        self.root.visited = True
        self.stack = Stack(size=graph.n_nodes)
        self.stack.stack_up(self.root)
    
    def search(self):
        top = self.stack.get_top()
        for neighbor in top.get_neighbors():
            if neighbor.value == top.value:
                continue
            
            if not neighbor.visited:
                neighbor.parent = top
                neighbor.level = top.level + 1
                neighbor.visited = True
                self.stack.stack_up(neighbor)
                self.search()
        self.stack.unstack()

In [30]:
dfs = DFS(graph, 1)

In [31]:
dfs.search()

In [33]:
for node in  dfs.graph.nodes[:5]:
    print("Node:", node.value)
    print("Parent:", node.parent)
    print("Level:", node.level)
    print("Visited:", node.visited)
    print("----")


Node: 1
Parent: None
Level: 0
Visited: True
----
Node: 2
Parent: 14
Level: 3
Visited: True
----
Node: 3
Parent: 1
Level: 1
Visited: True
----
Node: 4
Parent: 7
Level: 6
Visited: True
----
Node: 5
Parent: 2
Level: 4
Visited: True
----


In [ ]:
class Queue:

In [ ]:
from pyvis.network import Network

net = Network(notebook=True)

for node in graph.nodes:
    net.add_node(node.value, label=node.value)
    
for node in graph.nodes:
    for neighbor in node.neighbors:
        net.add_edge(node.value, neighbor.value)

net.show("graph.html")